In [ ]:
import kagglehub

# # Global AI Job Market Dataset (2010–2025)
# path = kagglehub.dataset_download("terencekatua/global-ai-job-market-dataset-20102025")
# print("Path to dataset files:", path)

# # AI & ML Job_Postings LinkedIn & Indeed (2025)
# path = kagglehub.dataset_download("ankit0017/ai-and-ml-job-postings-linkedin-and-indeed-2025")
# print("Path to dataset files:", path)

# #LinkedIn Job Postings (2023 - 2024)
# path = kagglehub.dataset_download("arshkon/linkedin-job-postings")
# print("Path to dataset files:", path)

c:\Users\user\PotenUP_AI\PJ02-DATA-ANALYSIS_Team4\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\user\.cache\kagglehub\datasets\terencekatua\global-ai-job-market-dataset-20102025\versions\1
Path to dataset files: C:\Users\user\.cache\kagglehub\datasets\ankit0017\ai-and-ml-job-postings-linkedin-and-indeed-2025\versions\1
Path to dataset files: C:\Users\user\.cache\kagglehub\datasets\arshkon\linkedin-job-postings\versions\13


In [3]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join('..')))
import src.utils as src

import pandas as pd

# 데이터 명세



### **01. Global AI Job Market Dataset (2010–2025)**


In [4]:
df = pd.read_csv("../data/global-ai-job-market_10-25/ai_impact_jobs_2010_2025.csv")

# 데이터셋의 형태 확인
df.shape

(5000, 22)

In [5]:
# 데이터 타입 확인 Non-Null count와 dtype을 통해 추론
# 핵심적인 numeric 변수는 sararly_usd

df.info()

<class 'pandas.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 22 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   job_id                              5000 non-null   str    
 1   posting_year                        5000 non-null   int64  
 2   country                             5000 non-null   str    
 3   region                              5000 non-null   str    
 4   city                                5000 non-null   str    
 5   company_name                        5000 non-null   str    
 6   company_size                        5000 non-null   str    
 7   industry                            5000 non-null   str    
 8   job_title                           5000 non-null   str    
 9   seniority_level                     5000 non-null   str    
 10  ai_mentioned                        5000 non-null   bool   
 11  ai_keywords                         1623 non-null   st

In [6]:
# int와 float 타입의 column만 선택
numeric_cols = df.select_dtypes(include=['int', 'float', 'str']).columns.tolist()

# 유니크 값 개수에 따라 분류
cat_columns = []
num_columns = []

for col in numeric_cols:
    if df[col].nunique() < 20:
        cat_columns.append(col)
    else:
        num_columns.append(col)

print("Categorical columns (< 20 unique values):", len(cat_columns))
print("Numerical columns (>= 20 unique values):", len(num_columns))
print("Neither categorical or numerical", set(df.columns) - set(cat_columns) - set(num_columns))

Categorical columns (< 20 unique values): 10
Numerical columns (>= 20 unique values): 10
Neither categorical or numerical {'ai_mentioned', 'reskilling_required'}


In [7]:
# 범주형 변수의 유니크 값 확인 
[print(i,':', len(df[i].unique())) for i in cat_columns]

# 종속성 확인
# city는 region과 개념적 종속성을 가지고 있음
# # 근데 여기선 랜덤하게 나옴
# 따라서 지역변수는 아무런 정보 가치가 없음

print(df.groupby('city')['region'].nunique())

# 
df[df['city'] == 'Bangalore']["region"].head()


posting_year : 16
region : 9
city : 14
company_name : 16
company_size : 5
industry : 9
job_title : 10
seniority_level : 6
ai_job_displacement_risk : 3
industry_ai_adoption_stage : 3
city
Bangalore    9
Berlin       9
Dubai        9
Lagos        9
London       9
Nairobi      9
New York     9
Paris        9
Sao Paulo    9
Seoul        9
Singapore    9
Sydney       9
Tokyo        9
Toronto      9
Name: region, dtype: int64


8     South Asia
16        Africa
17    South Asia
34        Africa
36        Europe
Name: region, dtype: str

In [8]:
# 수치형 변수의 유니크 값 확인
[print(i,':', len(df[i].unique()), df[i].dtype) for i in num_columns]
print("\n")
print("----"*20)
print("\n")

# 개념적인 범주형 변수가 존재함
cleaning_li = ['country', "ai_keywords", "core_skills","job_description_embedding_cluster"]
df.loc[df['ai_skills'].notna()][['job_title','ai_skills','core_skills', "ai_keywords"]].head()

# ai_skills와 _keywords는 거의 동일한 column >> 둘 중 하나는 삭제
src.check_dependency(df, "ai_skills", "ai_keywords")

job_id : 5000 str
country : 44 str
ai_keywords : 618 str
ai_intensity_score : 77 float64
core_skills : 4162 str
ai_skills : 618 str
salary_usd : 4883 int64
salary_change_vs_prev_year_percent : 1815 float64
automation_risk_score : 62 float64
job_description_embedding_cluster : 20 int64


--------------------------------------------------------------------------------


✅ 'ai_skills'은(는) 'ai_keywords'에 의해 결정됩니다. (1:1 또는 N:1 관계)


np.True_

In [19]:
# split skill


# 컬럼 정리
df["year"] = df["posting_year"].astype(int)
df["ai_mentioned"] = df["ai_mentioned"].astype(int)  # 0/1 가정



# =========================
# 2) core_skills split/explode + 연도별 카운트
# =========================
df["core_skills"] = df["core_skills"].fillna("")

skills = (
    df.assign(core_skill=df["core_skills"].str.split(","))
      .explode("core_skill")
)

skills["core_skill"] = skills["core_skill"].astype(str).str.strip()
skills = skills[skills["core_skill"] != ""]  # 빈 값 제거

skill_counts = (
    skills.groupby(["year", "core_skill"])
          .size()
          .reset_index(name="count")
)

# 전체에서 많이 등장한 Top-N 스킬만 추적 (라인 차트 가독성)
TOP_N = 10
top_skills = (
    skill_counts.groupby("core_skill")["count"]
               .sum()
               .sort_values(ascending=False)
               .head(TOP_N)
               .index
               .tolist()
)

skill_counts_top = skill_counts[skill_counts["core_skill"].isin(top_skills)]




# =========================
# 3) (추천) 스킬 변화는 '연도별 샘플 수' 영향을 받으니,
#    비율(share)로도 같이 보기
# =========================
# 연도별 총 스킬 토큰 수
total_skill_tokens_by_year = (
    skill_counts.groupby("year")["count"].sum().reset_index(name="total_tokens")
)

skill_share = skill_counts.merge(total_skill_tokens_by_year, on="year", how="left")
skill_share["share"] = skill_share["count"] / skill_share["total_tokens"]

skill_share_top = skill_share[skill_share["core_skill"].isin(top_skills)]


In [41]:
#skills["core_skill"]

df["core_skills"].unique()
#skill_counts

df[df["core_skills"] =='Research, Project Management, Business Analysis']

type(skills["core_skill"][0])

pandas.Series

### **02. AI & ML Job_Postings LinkedIn & Indeed (2025)**



In [105]:
# metadata
import json

with open('../data/raw/JobPosting_25(AIML)/ai-and-ml-job-postings-linkedin-and-indeed-2025-metadata.json', 'r') as f:
    metadata = json.load(f)

# 데이터셋 설명 출력
print(metadata['description'])

# 포함된 파일 목록 확인
for dist in metadata['distribution']:
    print(f"파일명: {dist.get('name')}, 형식: {dist.get('encodingFormat')}")

# AI & ML Job Postings Dataset — LinkedIn & Indeed (2025)

This dataset contains **time-stamped AI & ML job postings** scraped from **LinkedIn** and **Indeed** over multiple days, covering companies, roles, and locations. It includes:

- `link`: URL to the job posting
- `title`: Job title (e.g., Data Scientist, ML Engineer)
- `company`: Company name
- `location`: City, state, or country
- `date` & `time`: Job posting timestamp
- `scrape_date` & `scrape_time`: When the data was collected

**Dataset Highlights:**
- ~1,550 unique postings, clean and deduplicated
- Ready for **EDA, visualization, and ML experiments**
- Includes **scrape metadata** for temporal analysis

**Potential Use Cases:**
- Trend analysis of AI/ML hiring over time
- Skill extraction and NLP on job titles
- Job classification or predictive modeling projects
- Company hiring insights and labor market research
- Geospatial analysis of AI/ML demand

**Included Notebook:** `EDA_Job_Postings.ipynb`  
- Exploratory data ana

In [10]:
df2 = pd.read_csv("../data/JobPosting_25(AIML)/jobs_main.csv")
df2_merged_job = pd.read_csv("../data/JobPosting_25(AIML)/merged_jobs.csv")

# 데이터셋의 형태 확인
print(df2.shape)
print(df2_merged_job.shape)

(682, 6)
(1550, 8)


In [11]:
# 데이터 타입 확인 Non-Null count와 dtype을 통해 추론
# 핵심적인 numeric 변수는 sararly_usd

print(df2.info())
print(df2_merged_job.info())

<class 'pandas.DataFrame'>
RangeIndex: 682 entries, 0 to 681
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   job_url      682 non-null    str  
 1   title        682 non-null    str  
 2   company      682 non-null    str  
 3   location     487 non-null    str  
 4   scrape_date  682 non-null    str  
 5   scrape_time  682 non-null    str  
dtypes: str(6)
memory usage: 32.1 KB
None
<class 'pandas.DataFrame'>
RangeIndex: 1550 entries, 0 to 1549
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   link         1550 non-null   str  
 1   title        1550 non-null   str  
 2   company      1549 non-null   str  
 3   location     1311 non-null   str  
 4   date         1550 non-null   str  
 5   time         1550 non-null   str  
 6   scrape_date  1550 non-null   str  
 7   scrape_time  1550 non-null   str  
dtypes: str(8)
memory usage: 97.0 KB
None


In [12]:
print(df2_merged_job['scrape_date'].unique())
print(df2_merged_job['date'].unique())

print(df2_merged_job['scrape_time'].unique())
print(df2_merged_job['time'].unique())

# check Dependency == True
src.check_dependency(df2_merged_job, 'scrape_date', 'date')
src.check_dependency(df2_merged_job, 'scrape_time', 'time')


<StringArray>
['2025-09-18', '2025-09-19', '2025-09-20', '2025-09-21', '2025-09-22',
 '2025-09-23', '2025-09-24', '2025-09-25', '2025-09-26', '2025-09-27',
 '2025-09-28', '2025-09-29', '2025-09-30', '2025-10-01', '2025-10-02',
 '2025-10-03', '2025-10-04', '2025-10-05', '2025-10-06']
Length: 19, dtype: str
<StringArray>
['2025-09-18', '2025-09-19', '2025-09-20', '2025-09-21', '2025-09-22',
 '2025-09-23', '2025-09-24', '2025-09-25', '2025-09-26', '2025-09-27',
 '2025-09-28', '2025-09-29', '2025-09-30', '2025-10-01', '2025-10-02',
 '2025-10-03', '2025-10-04', '2025-10-05', '2025-10-06']
Length: 19, dtype: str
<StringArray>
['16:44:51', '17:18:02', '18:30:15', '18:50:50', '01:38:14', '03:47:59',
 '05:18:55', '05:40:14', '06:30:33', '07:19:31',
 ...
 '01:38:46', '04:24:27', '04:42:30', '05:19:58', '05:40:25', '06:32:03',
 '07:20:51', '08:48:24', '09:24:43', '10:23:15']
Length: 426, dtype: str
<StringArray>
['16:44:51', '17:18:02', '18:30:15', '18:50:50', '01:38:14', '03:47:59',
 '05:18:55',

np.True_

In [13]:
# merge 2 dataframes
temp = df2.copy()
temp2 = df2_merged_job.copy()

temp.rename(columns={'job_url': 'url'}, inplace=True)
temp2.rename(columns={'link': 'url'}, inplace=True)

temp.head()
df_merged = pd.concat([temp, temp2], ignore_index=True)
print(df_merged.shape)
print(df_merged.info())

df_merged.drop(['date','time'], axis=1, inplace=True)
df_merged.shape

(2232, 8)
<class 'pandas.DataFrame'>
RangeIndex: 2232 entries, 0 to 2231
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   url          2232 non-null   str  
 1   title        2232 non-null   str  
 2   company      2231 non-null   str  
 3   location     1798 non-null   str  
 4   scrape_date  2232 non-null   str  
 5   scrape_time  2232 non-null   str  
 6   date         1550 non-null   str  
 7   time         1550 non-null   str  
dtypes: str(8)
memory usage: 139.6 KB
None


(2232, 6)

In [44]:
#save merged dataframe
df_merged.to_csv("../data/jobs_merged_0129.csv", index=False)

In [110]:
#read this check point
df2 = pd.read_csv("../data/df2_0129.csv")


In [111]:
df2.describe()
# url unique unmatched
# ["company"] freq == 284
# location??

,url,title,company,location,scrape_date,scrape_time
count,2232,2232,2231,1798,2232,2232
unique,2129,758,967,106,26,600
top,https://www.linkedin.com/jobs/view/4307968830,Remote Python AI Engineer - 17852,Turing,"Bengaluru, Karnataka, India",2025-07-16,13:55:55
freq,10,149,284,434,298,31


In [ ]:
df2['country'] = df2["location"].str.split(',').apply(lambda x: x[-1].strip() if isinstance(x, list) and len(x) > 0 else '')
print(df2[df2['country'] == "South Korea"])

#idx : 1682 가짜 공고 인듯?
# 모든 공고가 인도에 있음

,url,title,company,location,scrape_date,scrape_time,country
1682,https://www.linkedin.com/jobs/view/4306697434,머신러닝 엔지니어 / Machine Learning Engineer,Drift Wave,"Seoul, South Korea",2025-09-30,06:31:44,South Korea


In [109]:
# remote를 포함하는 job의 특성 문해
df2[df2['title'].str.contains('remote', case=False, na=False)]["company"].value_counts() # (234, )



company
Turing                                                   154
Uplers                                                    32
BairesDev                                                 21
Perimattic                                                 4
Jobgether                                                  3
iBovi - Staffing, Consulting and Recruitment Services      2
Seventh Contact Hiring Solutions                           2
Call For Referral                                          2
VGreen Technology Solutions (VGreenTEK)                    1
h3 Technologies, LLC                                       1
Smart Working                                              1
Live Nation Entertainment                                  1
HYI.AI                                                     1
BotifyNow AI                                               1
MARICI Solutions GmbH                                      1
Crossing Hurdles                                           1
Arrise Solutions

### **03. LinkedIn Job Postings (2023 - 2024)**

# Utils Func

In [16]:
def classify_columns(df, threshold=20):
    """
    데이터프레임의 컬럼을 범주형(cat)과 수치형(num)으로 자동 분류합니다.
    ID나 Index 성격의 컬럼은 수치형에서 제외합니다.
    """
    # 1. 대상 컬럼 선택 (수치형 + 문자열)
    # str은 pandas에서 보통 'object' 타입이므로 'object'를 포함합니다.
    target_cols = df.select_dtypes(include=['int', 'float', 'str', 'object']).columns.tolist()

    cat_columns = []
    num_columns = []
    index_columns = [] # 인덱스성 컬럼을 따로 관리하면 나중에 확인하기 좋습니다.

    for col in target_cols:
        # A. 인덱스/ID 컬럼 필터링 (이름에 id/index가 있거나, 모든 값이 유니크할 때)
        is_index_name = any(ext in col.lower() for ext in ['id', 'index', 'idx', '_no'])
        is_high_cardinality = (df[col].nunique() == len(df))
        
        if is_index_name or is_high_cardinality:
            index_columns.append(col)
            continue # index 리스트에 넣었으므로 다음 루프로 넘어감

        # B. 유니크 값 개수에 따른 분류
        if df[col].nunique() < threshold:
            cat_columns.append(col)
        else:
            num_columns.append(col)

    print(f"✅ 분류 완료: 범주형({len(cat_columns)}), 수치형({len(num_columns)}), 제외된 인덱스({len(index_columns)})")
    
    return cat_columns, num_columns, index_columns

# 사용 예시
cat_list, num_list, idx_list = classify_columns(df)


✅ 분류 완료: 범주형(10), 수치형(9), 제외된 인덱스(1)


In [17]:
def check_dependency(df, child_col, parent_col):
    """
    child_col(예: city)이 parent_col(예: region)에 완전히 종속되는지 확인합니다.
    """
    # child_col 값 하나당 parent_col 값이 몇 종류인지 계산
    counts = df.groupby(child_col)[parent_col].nunique()
    
    # 모든 child_col에 대해 parent_col 종류가 1개뿐인지 확인
    is_dependent = (counts <= 1).all()
    
    if is_dependent:
        print(f"✅ '{child_col}'은(는) '{parent_col}'에 의해 결정됩니다. (1:1 또는 N:1 관계)")
    else:
        # 관계가 깨지는 데이터 추출
        conflicts = counts[counts > 1]
        print(f"❌ '{child_col}' 중 일부가 여러 '{parent_col}'에 속해 있습니다.")
        print(f"--- 위반 사례 (처음 5개) ---\n{conflicts.head()}")
        
    return is_dependent

# 사용 예시:
check_dependency(df, 'city', 'region')

❌ 'city' 중 일부가 여러 'region'에 속해 있습니다.
--- 위반 사례 (처음 5개) ---
city
Bangalore    9
Berlin       9
Dubai        9
Lagos        9
London       9
Name: region, dtype: int64


np.False_